# Kiểm tra và xóa các file audio không có trong validated

In [ ]:
import os

data_dir = "C:\ComSL\swedish_dataset" # sửa path 
clips_dir = os.path.join(data_dir, "clips")

tsv_files = [
    "validated.tsv"
]

print("Số lượng dòng trong các file tsv:")
for tsv_file in tsv_files:
    tsv_path = os.path.join(data_dir, tsv_file)  
    try:
        with open(tsv_path, 'r', encoding='utf-8') as f:
            line_count = sum(1 for line in f)
        print(f"{tsv_file}: {line_count}")
    except Exception as e:
        print(f"{tsv_file}: Lỗi - {e}")

audio_extensions = ('.mp3', '.wav', '.flac', '.ogg')
audio_count = sum(1 for f in os.listdir(clips_dir) if f.lower().endswith(audio_extensions))
print(f"\nSố lượng audio trong folder clips: {audio_count}")

Số lượng dòng trong các file tsv:
validated.tsv: 5899

Số lượng audio trong folder clips: 7064


In [7]:
tsv_files = [
    "covost_v2.sv-SE_en.tsv" # sửa path 
]

print("Số lượng dòng trong các file tsv:")
for tsv_file in tsv_files:
    tsv_path = os.path.join(data_dir, tsv_file)  
    try:
        with open(tsv_path, 'r', encoding='utf-8') as f:
            line_count = sum(1 for line in f)
        print(f"{tsv_file}: {line_count}")
    except Exception as e:
        print(f"{tsv_file}: Lỗi - {e}")

Số lượng dòng trong các file tsv:
covost_v2.sv-SE_en.tsv: 5899


In [10]:
import os
import csv

data_dir = r"C:\ComSL\swedish_dataset" # sửa path
clips_dir = os.path.join(data_dir, "clips")
validated_tsv = os.path.join(data_dir, "validated.tsv")

# Đọc tất cả path hợp lệ từ validated.tsv
valid_paths = set()
with open(validated_tsv, 'r', encoding='utf-8') as tsvfile:
    reader = csv.DictReader(tsvfile, delimiter='\t')
    for row in reader:
        valid_paths.add(row['path'])

# Duyệt tất cả file trong clips, xoá file không có trong valid_paths
deleted_count = 0
for file_name in os.listdir(clips_dir):
    if file_name not in valid_paths:
        file_path = os.path.join(clips_dir, file_name)
        try:
            os.remove(file_path)
            deleted_count += 1
        except Exception as e:
            print(f"Lỗi xoá {file_name}: {e}")

print(f"Đã xoá {deleted_count} file không có trong validated.tsv.")

Đã xoá 0 file không có trong validated.tsv.


# Scale train/dev/test

In [2]:
import pandas as pd

In [11]:
tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.train.tsv" # sửa path
df = pd.read_csv(tsv_path, sep="\t")

# Giữ lại 1200 dòng đầu tiên
train_df = df.head(1200)

# Lưu ra file mới (ví dụ ở thư mục working, hoặc ghi đè nếu muốn)
train_tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.train.tsv" # sửa path
train_df.to_csv(train_tsv_path, sep="\t", index=False)

print(f"Đã tạo file: {train_tsv_path} với {len(train_df)} dòng")
len(train_df)

Đã tạo file: C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.train.tsv với 1200 dòng


1200

In [13]:
tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.test.tsv" # sửa path
df = pd.read_csv(tsv_path, sep="\t")

# Giữ lại 150 dòng đầu tiên
test_df = df.head(150)

test_tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.test.tsv"
test_df.to_csv(test_tsv_path, sep="\t", index=False)

print(f"Đã tạo file: {test_tsv_path} với {len(test_df)} dòng")
len(test_df)

Đã tạo file: C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.test.tsv với 150 dòng


150

In [12]:
tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.dev.tsv" # sửa path
df = pd.read_csv(tsv_path, sep="\t")

# Giữ lại 150 dòng đầu tiên
dev_df = df.head(150)

# Lưu ra file mới (ví dụ ở thư mục working, hoặc ghi đè nếu muốn)
dev_tsv_path = "C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.dev.tsv"
dev_df.to_csv(dev_tsv_path, sep="\t", index=False)

print(f"Đã tạo file: {dev_tsv_path} với {len(dev_df)} dòng")
len(dev_df)

Đã tạo file: C:\ComSL\swedish_dataset\covost_v2.sv-SE_en.dev.tsv với 150 dòng


150

# Kaggle - Visualize 

%pip install tensorboard

In [ ]:
import os

for file in os.listdir("/kaggle/working/comsl/output/comsl_mn2en/logs/ComST/default"):
    print(file)

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os

log_dir = "/kaggle/working/comsl/output/comsl_mn2en/logs/ComST/default"
# Chọn file events lớn nhất/thường là mới nhất
event_files = [f for f in os.listdir(log_dir) if f.startswith('events.out')]
event_path = os.path.join(log_dir, sorted(event_files)[-1])

event_acc = EventAccumulator(event_path)
event_acc.Reload()

print("Tất cả các scalar tag:")
print(event_acc.Tags()['scalars'])  # In ra các metric mà bạn có thể đọc

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/comsl/output/comsl_mn2en/logs/ComST/default

In [ ]:
train_acc = event_acc.Scalars('train_acc_asr')
train_acc

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


steps = np.array([e.step for e in train_acc])
values = np.array([e.value for e in train_acc])

num_samples = 1200
batch_size = 10 
num_epochs = 5
steps_per_epoch = int(np.ceil(num_samples / batch_size))

# Map step sang epoch
epoch_ids = (steps // steps_per_epoch).astype(int)   # epoch bắt đầu từ 0

# Lưu trữ accuracy cho từng epoch
epoch_acc_dict = {}
for epoch in range(num_epochs):
    epoch_acc = values[epoch_ids == epoch]
    if len(epoch_acc) > 0:
        epoch_acc_dict[epoch+1] = epoch_acc.mean()  # lấy giá trị trung bình mỗi epoch
    else:
        epoch_acc_dict[epoch+1] = np.nan  # nếu epoch này chưa có dữ liệu

# Convert về array để vẽ
epochs = np.array(list(epoch_acc_dict.keys()))
epoch_accs = np.array(list(epoch_acc_dict.values()))

# Plot
plt.plot(epochs, epoch_accs, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Train Accuracy (%)")
plt.title("Train Accuracy per Epoch")
plt.grid(True)
plt.show()

# Kaggle - download checkpoint